## Itération simultanée pour le calcul de valeurs propres

On veut implémenter une méthode itérative pour calculer simultanément les $p \leq n$ valeurs propres dominantes d'une matrice $A\in \mathbb C^{n\times n}$, qu'on supposera Hermitienne.
La méthode s'appuie sur la décomposition QR réduite, de la forme:

$$ QR = M,$$
où:
- $M\in\mathbb C^{n\times p}$.
- $Q\in \mathbb C^{n\times p}$ et ses colonnes forment une famille orthonormée de $\mathbb C^n$ pour le produit scalaire usuel.
- $R\in \mathbb C^{p\times p}$ est une matrice triangulaire supérieure avec des entrées diagonales réelles et positives.

#### Question 1
Donner la décomposition QR réduite de $M$ si $p=1$.

#### Question 2
En écrivant
$$ Q = \begin{pmatrix} \widetilde Q &\mathbf{q}\end{pmatrix},\qquad R = \begin{pmatrix} \widetilde R & \mathbf{r} \\ 0 & \alpha\end{pmatrix}, \qquad M = \begin{pmatrix} \widetilde{M} & \mathbf m\end{pmatrix}$$
avec $\widetilde Q,\widetilde M \in \mathbb C^{n\times (p-1)}$, $\widetilde R\in\mathbb C^{(p-1)\times (p-1)}$, $\mathbf r \in \mathbb C^{p-1}$, $\alpha\in \R_+$ et $\mathbf m\in \mathbb C^n$, donner la décomposition QR de $M$ sachant la décomposition $\widetilde Q \widetilde R = \widetilde M$ de la matrice formée des $(p-1)$ premières colonnes de $M$. (Utiliser les propriétés de la décomposition QR).

#### Question 3
 Implémenter une méthode pour calculer la décomposition QR réduite.

**Remarque.**
Par souci d'efficacité,
on peut utiliser la macro `@views`,
qui permet d'effectuer des opérations avec des sous-matrices sans en copier le contenu. (voir `?view` et `?@views` dans le REPL).
Voir [cette partie](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-views) de la documentation pour plus d'informations.
Nous reprenons ci-dessous l'exemple de la page du manuel pour illustrer ce point.

In [ ]:
using BenchmarkTools
x = rand(10^6)
fcopy(x) = sum(x[2:end-1])
@views fview(x) = sum(x[2:end-1])
t1 = @belapsed fcopy(x)
t2 = @belapsed fview(x)
println(t1, " ", t2)

# The following code may be helpful to develop your understanding
a = zeros(3)
@views b = a[1:2]  # Does not copy a[1:2]
c = a[2:3]         # Does copy a[2:3]
a[2] = 1.
println(b, " ", c)

In [ ]:
using LinearAlgebra

@views function myQR(M)
    n,p = size(M)

    @assert p <= n "Error: p > n"
    ## implémenter la décomposition QR ici
    Q = zero(M)
    R = zeros(eltype(M),p,p)

    return Q,R
end

In [ ]:
M = randn(ComplexF64,100,50)
M = Complex{BigFloat}.(M)
Q,R = myQR(M)

@show norm(Q'Q-I) # vérifier que Q a bien des colonnes orthonormales
@show all(isreal.(diag(R))) # R a des coefficients diagonaux réels
@show all(real.(diag(R)) .>= 0) # et positifs
@show norm(Q*R-M) # et que QR = M à une erreur d'arrondi près


In [ ]:
@time qr(M) #implémentation de LinearAlgebra
@time myQR(M) #implémentation maison

#### Question 4
La méthode d'itération simultanée est définie, étant donné une condition initiale $X_0\in\mathbb C^{n\times p}$, par la récurrence:
$$ X_{n+1} R_{n+1} = A X_n,$$
où $X_{n+1} R_{n+1}$ est la décomposition QR réduite de $AX_n \in \mathbb C^{n\times p}$. Il est possible de montrer que pour $A$ Hermitienne, et sous certaines conditions initiales $X_0$ et les $p$ premières valeurs propres de $A$ de plus grand module, $X_n$ converge essentiellement colonne par colonne vers la matrice formée par les $p$ vecteurs propres associés (voir les notes de cours pour une preuve).

Implémenter une fonction `myEigen` en Julia prenant en arguments : une matrice `M` (supposée Hermitienne), un entier `nev` (le nombre de valeurs propres à calculer), un entier `niter` (le nombre d'itérations), et une matrice initiale `X₀`. La fonction devrait renvoyer un vecteur de `nev` valeurs propres et une matrice formée d'une approximation des `nev` vecteurs propres dominants de `M` en colonnes.

Pour des matrices dont les entrées ne sont pas des `BigFloat`, on pourra comparer les résultats ceux donnés par `LinearAlgebra.eigen` pour débugger.

In [ ]:
n,nev = 10,3

## seed pour le RNG
using Random
Random.seed!(2023)
##

X = randn(n,n)/sqrt(n)
X₀ = randn(n,nev)/sqrt(n)


function myEigen(M,nev,niter,X₀)
    ## implémenter l'itération ici
    return λs,us
end

M = BigFloat.(I + X'X)

@time λs,us=myEigen(M,nev,500,X₀)

display(λs)
display(us)
display(M*us - λs' .*us)

#### Question 5

Étudier numériquement la vitesse de convergence des 3 premiers vecteurs propres vers leur limite en fonction du nombre d'itérations de la méthode. On pourra considérer que tous les vecteurs propres sont convergés après un grand nombre (disons 10000) itérations pour obtenir des valeurs de référence. L'écart entre deux vecteurs $u$ et $v$ sera mesuré en terme de l'angle formé par les sous-espaces engendrés:
$$ \theta_{u,v} = \arccos\left(\frac{|u^\intercal v|}{\|u\|\|v\|}\right).$$
Comparer l'erreur commise à la $k$-ème itération et les quantités $(\lambda_i/\lambda_{i+1})^{-k},\, i=1,2,3$, sur un graphe avec une échelle log en ordonnée.

Pour garantir numériquement que $0\leq \frac{|u^\intercal v|}{\|u\|\|v\|}\leq 1$, on pourra utiliser la fonction `clamp`.

In [ ]:
@time λs,us=myEigen(M,n,10000,randn(n,n)) # on calcule les valeurs et vecteurs propres "convergés"
display(M*us - λs' .*us)

In [ ]:
nev=3

iters = 1:200
errs = [Float64[] for i=1:nev]

for niter in iters
    λhats,uhats = myEigen(M,nev,niter,X₀)
    ## calculer les erreurs ici
end

using Plots,LaTeXStrings
colors = (:red,:blue,:green)
pl =plot(xlabel="Itération k", ylabel="Erreur essentielle",yaxis=:log,legend=:bottomleft) # on déclare un plot

for i=1:nev
    plot!(pl,iters,errs[i],label=L"θ_{%$i}",color=colors[i])
    plot!(pl,k->(λs[i]/λs[i+1])^-k,label=L"(λ_{%$i}/λ_{%$(i+1)})^{-k}",linestyle=:dot,color=colors[i])
end

pl

# Algorithme PageRank

PageRank est un algorithme permettant d'attribuer un score aux sommets d'un graphe orienté.
Il est utilisé par de nombreux moteurs de recherche pour trier les résultats de recherche.
Dans ce contexte, le graphe orienté encode les liens entre les pages du World Wide Web :
les sommets du graphe orienté représentent des pages web,
et les arêtes représentent les connexions entre les pages:
il y a une arête allant de la page $i$ à la page $j$ si la page $i$ contient un lien hypertexte vers la page $j$.

Considérons un graphe orienté $G(V, E)$ avec des sommets $V = \{1, \dotsc, n\}$ et des arêtes $E$.
Le graphe peut être représenté par sa matrice d'adjacence $\mathsf A \in \{0, 1\}^{n \times n}$,
dont les entrées sont données par
$$
a_{ij} =
\begin{cases}
    1 & \text{si il y a une arête de $i$ à $j$,} \\
    0 & \text{sinon.}
\end{cases}
$$
L'idée de l'algorithme PageRank, dans sa forme la plus simple,
consiste à attribuer des scores $r_i$ aux sommets
en résolvant le système d'équations suivant :
$$ \tag{PageRank}
    \forall i \in  V, \qquad
    r_i
    = \sum_{j \in \mathcal N(i)} \frac{r_j}{o_j}.
$$
<span id="pagerank"></span>
où $o_j$ est le degré sortant du sommet $j$,
c'est-à-dire le nombre d'arêtes ayant $j$ pour origine.
Ici, la somme s'applique à l'ensemble des nœuds dans $\mathcal N(i)$,
qui représente l'ensemble des voisins entrants du sommet $i$,
c'est-à-dire ceux qui ont une arête pointant vers le sommet $i$.

Soit $\mathbf r = \begin{pmatrix} r_1 & \dots & r_n \end{pmatrix}^T$.
Il est simple de montrer que résoudre le système <a href="#pagerank">(PageRank)</a> revient à résoudre le problème suivant:
$$  \tag{PageRank-vector}
    \mathbf r =
    \mathsf A^T
    \begin{pmatrix}
        \frac{1}{o_1} & &  \\
                      & \ddots & \\
                      & & \frac{1}{o_n}
    \end{pmatrix}
    \mathbf r =:  \mathsf A^T \mathsf O^{-1} \mathbf r.
$$
<span id="pagerank"></span>
En d'autres termes,
le problème revient à trouver un vecteur propre de valeur propre $1$ de la matrice $\mathsf M = \mathsf A^T \mathsf O^{-1}$.
Notons qu'à ce stade, nous n'avons prouvé ni l'existence,
ni l'unicité d'une solution de cette équation.
La question de l'unicité d'une solution est liée à la connectivité du graphe et ne sera pas abordée ici.
Nous allons démontrer, par contre, qu'il existe une solution au problème.

**Remarque.** La matrice $\mathsf O^{-1} \mathsf A$ est la matrice de transition d'une marche aléatoire sur le graphe orienté,
où à chaque pas un déplacement se fait vers un voisin sortant,
avec une probabilité égale pour chacun d'eux.
Résoudre <a href="#pagerank">(PageRank-vector)</a> revient à trouver une distribution stationaire de cette marche aléatoire.

**Questions:**

1. Remarquer que $\mathsf M$ est une matrice stochastique gauche,
   c'est-à-dire que la somme des éléments de chaque colonne est égale à 1.

1. Prouver que les valeurs propres de n'importe quelle matrice $\mathsf B \in \mathbb R^{n \times n}$ coïncident avec celles de $\mathsf B^T$.
   Vous pouvez utiliser le fait que $\det(\mathsf B) = \det(\mathsf B^T)$.

1. En utilisant les points précédents, montrer que 1 est une valeur propre et que $\rho(\mathsf M) = 1$.
   Pour la deuxième partie, trouver une norme matricielle subordonnée telle que $\lVert\mathsf M\rVert= 1$.
   Ceci démontre l'existence d'une solution de <a href="#pagerank">(PageRank-vector)</a>,
   et prouve aussi que 1 est la valeur propre dominante de $\mathsf M$

1. Implémenter sans utiliser le package `SparseArrays` l'algorithme PageRank afin de classer les pages de l'encyclopédie Wikipedia anglophone telle qu'elle était en 2013
   (voir indication ci-dessous pour le téléchargement du jeu de données).
   Après avoir attribué un score à toutes les pages,
   imprimer les 10 pages les mieux classées,
   ce qui devrait donner ceci:
   ```
   United States, United Kingdom, World War II, Latin, France, Germany, English language, China, Canada, India
   ```

1. Écrire une fonction `search(keyword)` permettant d'effectuer une recherche dans la base de données.
   Voici un exemple de ce que cette fonction pourrait renvoyer :
   ```
   julia> search("Newton")
   47-element Vector{String}:
    "Isaac Newton"
    "Newton (unit)"
    "Newton's laws of motion"
    …
   ```

**Indications supplémentaires**:

- Le code ci-dessous permet de télécharger le jeu de données.

In [ ]:
using LinearAlgebra
import Downloads
import Tar

# URL where data can be downloaded
url = "https://urbain.vaes.uk/static/wikidata.tar"

# Download the data
filename = "wikidata.tar"
isfile(filename) || Downloads.download(url, filename)

# Extract data into directory `wikidata`
dirname = "wikidata"
isdir(dirname) || Tar.extract(filename, "wikidata")

Ce jeu de données contient un sous-ensemble des données disponibles publiquement <a href="https://snap.stanford.edu/data/enwiki-2013.html">ici</a>.
Afin de limiter le temps de calcul, seuls 5% des articles les mieux notés ont été conservés pour cet exercice.
Une fois l'archive décompressée,
le jeu de données peut être chargé dans Julia de la manière suivante :

In [ ]:
import CSV
import DataFrames

# Read nodes and edges into data frames
nodes_dataframe = CSV.read("$dirname/names.csv", DataFrames.DataFrame)
edges_dataframe = CSV.read("$dirname/edges.csv", DataFrames.DataFrame)

# Convert data to matrices
nodes = Matrix(nodes_dataframe)
edges = Matrix(edges_dataframe)

# The data structures should be self-explanatory
edges_dataframe

- Vous pouvez utiliser soit la version simplifiée de l'algorithme donnée en <a href="#pagerank">(PageRank-vector)</a>,
  soit la version améliorée avec un facteur d'amortissement décrite sur Wikipedia.

- Comme la valeur propre dominante est connue *a priori*,
  le critère d'arrêt suivant pourra être utilisé:
  $$
      \frac{\lVert\mathsf M \widehat{\mathbf r} - \widehat{\mathbf r}\rVert_1}{\lVert\widehat{\mathbf r}\rVert_1} < 10^{-10}.
  $$
  où $\widehat{\mathbf r}$ est une approximation du vecteur propre correspondant à la valeur propre dominante.

- Toutes les méthodes de résolution de <a href="#pagerank">(PageRank-vector)</a> nécessitent de calculer des produits matrice-vecteur avec la matrice $\mathsf M$.
  Cette matrice étant très grande,
  elle ne peut pas être stockée comme une matrice dense de type `Matrix`.
  Pour résoudre le problème, il est donc nécessaire de définir une structure de matrice creuse.
  On pourra par exemple utiliser le format de matrice creuse COO,
  qui est le plus simple.
  Une autre option est d'utiliser le format CSR,
  qui permet de calculer les produits matrice-vecteur de manière plus efficace.
  Il sera utile, pour résoudre le problème aux valeurs propres,

    * de définir la méthode de multiplication matrice-vecteur `*(A::my_sparse_matrix, b::Vector{Float64})`.

    * de définir une fonction `my_sparse` permettant de construire une matrice creuse à partir de vecteurs `rows`, `cols` et `vals` au format COO.
      Évidemment, si vous choisissez d'utiliser le format COO,
      cette fonction est triviale.

In [ ]:
# Necessary to overload `*`
import Base: *

# Modify if necessary
struct my_sparse_matrix
    rows::Vector{Int}
    cols::Vector{Int}
    vals::Vector{Float64}
    m::Int
    n::Int
end

function *(A::my_sparse_matrix, b::Vector{Float64})
    # Fill me
end

# Constructor from parameters in COO format (modify if necessary)
function my_sparse(rows, cols, vals, m, n)
    my_sparse_matrix(rows, cols, vals, m, n)
end

# Test the code
m, n = 4, 3
R = [2, 2, 2, 3, 3]
C = [1, 2, 3, 1, 3]
V = [5., 6., 7., 8., 9.]
A = my_sparse(R, C, V, m, n)
b = [1.; 1.; 1.]
@assert A*b == [0.; 18.; 17.; 0.] "Multiplication does not work!"

**Remarque.** Bien qu'il soit demandé de ne pas utiliser `SparseArrays` dans votre code final,
l'utilisation de ce package pour vos tests est encouragée.
Il peut être utile, par exemple, d'implémenter PageRank d'abord avec une matrice creuse construite avec `SparseArrays`,
avant d'utiliser votre propre structure de matrice creuse.
Pour rappel, une matrice creuse peut être construite à partir des paramètres du format COO de la manière suivante avec `SparseArrays`:

In [ ]:
import SparseArrays
A = SparseArrays.sparse(R, C, V, m, n)
@assert A*b == [0.; 18.; 17.; 0.] "Multiplication does not work!"

Nous vous proposons de découper le reste de votre code en trois cellulles:

- Construire la matrice $\mathsf M$:

In [ ]:
# Fill me

- Résoudre le problème PageRank (la fonction `sortperm` peut être utile ici) et imprimer les noms des 10 pages les mieux classées:

In [ ]:
# Fill me

- Définir et tester la fonction `search`
(les fonctions `filter` et `occursin` peuvent être utiles ici):

In [ ]:
function search(keyword)
    # Fill me
end
search("Newton")